## Time Out

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.timeout.es"
MAIN_URL = BASE_URL + "/madrid/es/que-hacer"

headers = {
    "User-Agent": "Mozilla/5.0"
}

def get_event_links():
    res = requests.get(MAIN_URL, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    cards = soup.find_all("a", href=True)

    # Filtrar solo URLs de eventos
    event_links = [BASE_URL + a["href"] for a in cards if "/madrid/es/que-hacer/" in a["href"]]
    return list(set(event_links))  # Eliminar duplicados

def parse_event(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    def safe_text(selector):
        tag = soup.select_one(selector)
        return tag.get_text(strip=True) if tag else "N/A"

    nombre_evento = safe_text("h1, h2")
    descripcion = soup.get_text(" ", strip=True).lower()

    # Reglas básicas por palabras clave
    discapacidad = "visual" if "lengua de signos" in descripcion else "ninguna"
    modalidad = "exterior" if "aire libre" in descripcion else "interior"
    costo = "gratis" if "gratis" in descripcion else "pago"

    # Valores manuales o heurísticos
    edad_dirigida = "todas las edades" if "familiar" in descripcion or "niños" in descripcion else "N/A"
    min_integrantes = "N/A"
    ubicacion = safe_text("[data-testid*=location]") or "N/A"
    categoria = safe_text("ul.breadcrumbs li:nth-last-child(1)") or "N/A"

    return {
        "nombre_evento": nombre_evento,
        "categoría": categoria,
        "discapacidad": discapacidad,
        "ubicación": ubicacion,
        "costo": costo,
        "edad_dirigida": edad_dirigida,
        "min_integrantes": min_integrantes,
        "modalidad": modalidad,
        "url": url
    }

# Extraer eventos
eventos = []
for link in get_event_links()[:10]:  # Ajusta el número de eventos aquí
    try:
        print(f"Procesando: {link}")
        evento = parse_event(link)
        eventos.append(evento)
    except Exception as e:
        print(f"Error en {link}: {e}")

# Crear DataFrame
df = pd.DataFrame(eventos)

# Mostrar DataFrame
df

# Opcional: Guardar en CSV
#df.to_csv("eventos_timeout_madrid.csv", index=False)


Procesando: https://www.timeout.es/madrid/es/que-hacer/5-cosas-para-hacer-hoy-en-madrid
Procesando: https://www.timeout.es/madrid/es/que-hacer/camas-elasticas-y-parques-de-bolas-juegos-de-ninyos-para-adultos
Procesando: https://www.timeout.es/madrid/es/que-hacer/dias-festivos-y-principales-eventos-de-2014-en-madrid
Procesando: https://www.timeout.es/madrid/es/que-hacer/10-cosas-para-hacer-esta-semana-en-madrid
Procesando: https://www.timeout.es/madrid/es/que-hacer/que-hacer-en-madrid-abril
Procesando: https://www.timeout.es/madrid/es/que-hacer/que-hacer-este-fin-de-semana-en-madrid
Procesando: https://www.timeout.es/madrid/es/que-hacer/circulo-de-bellas-artes


In [4]:
df

,nombre_evento,categoría,discapacidad,ubicación,costo,edad_dirigida,min_integrantes,modalidad,url
0,Qué hacer hoy en Madrid,N/A,ninguna,Lavapiés,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/5-c...
1,Camas elásticas y parques de bolas: sitios par...,N/A,ninguna,N/A,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/cam...
2,Eventos y días festivos en Madrid este 2025,N/A,ninguna,N/A,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/dia...
3,10 planes para hacer en Madrid esta semana,N/A,ninguna,Legazpi,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/10-...
4,Qué hacer en Madrid en abril,N/A,ninguna,Madrid,gratis,todas las edades,N/A,exterior,https://www.timeout.es/madrid/es/que-hacer/que...
5,Qué hacer este fin de semana en Madrid,N/A,ninguna,Madrid,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/que...
6,Círculo de Bellas Artes,N/A,ninguna,N/A,gratis,todas las edades,N/A,interior,https://www.timeout.es/madrid/es/que-hacer/cir...


## Que hacer con los niños 

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL principal
BASE_URL = "https://quehacerconlosninos.es"
OCIO_URL = f"{BASE_URL}/ocio/"

# Encabezados para la solicitud HTTP
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Función para obtener los enlaces de los artículos
def obtener_enlaces_articulos():
    res = requests.get(OCIO_URL, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    enlaces = []

    # Buscar todos los enlaces de artículos en la página
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/"):
            href = BASE_URL + href
        if "/ocio/" in href and href not in enlaces:
            enlaces.append(href)
    return enlaces

# Función para extraer información de cada artículo
def extraer_info_articulo(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Extraer el título del artículo
    titulo_tag = soup.find("h1")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "N/A"

    # Extraer la categoría (si está disponible)
    categoria_tag = soup.find("a", {"rel": "category tag"})
    categoria = categoria_tag.get_text(strip=True) if categoria_tag else "N/A"

    # Extraer la ubicación (si está disponible)
    ubicacion = "Madrid"  # Asumimos que la mayoría de los eventos son en Madrid

    # Extraer el costo (si se menciona "gratis" en el texto)
    texto = soup.get_text().lower()
    costo = "Gratis" if "gratis" in texto else "Pago"

    # Extraer la edad dirigida (si se menciona en el texto)
    if "niños" in texto or "familia" in texto:
        edad_dirigida = "Niños/Familia"
    else:
        edad_dirigida = "N/A"

    # Modalidad (interior o exterior) basada en palabras clave
    if "aire libre" in texto or "parque" in texto:
        modalidad = "Exterior"
    else:
        modalidad = "Interior"

    # Discapacidad (si se menciona accesibilidad en el texto)
    if "accesible" in texto or "silla de ruedas" in texto:
        discapacidad = "Accesible"
    else:
        discapacidad = "N/A"

    # Mínimo de integrantes (si se menciona en el texto)
    min_integrantes = "N/A"  # No se especifica en la mayoría de los casos

    return {
        "nombre_evento": titulo,
        "categoría": categoria,
        "discapacidad": discapacidad,
        "ubicación": ubicacion,
        "costo": costo,
        "edad_dirigida": edad_dirigida,
        "min_integrantes": min_integrantes,
        "modalidad": modalidad,
        "url": url
    }

# Obtener los enlaces de los artículos
enlaces_articulos = obtener_enlaces_articulos()

# Extraer información de cada artículo
eventos = []
for enlace in enlaces_articulos[:10]:  # Limitar a los primeros 10 artículos
    print(f"Procesando: {enlace}")
    info = extraer_info_articulo(enlace)
    eventos.append(info)

# Crear un DataFrame con la información recopilada
df1= pd.DataFrame(eventos)

# Mostrar el DataFrame
df1

# Opcional: Guardar el DataFrame en un archivo CSV
#df1.to_csv("eventos_quehacerconlosninos.csv", index=False)


Procesando: https://quehacerconlosninos.es/ocio/
Procesando: https://quehacerconlosninos.es/ocio/?e-page-499d516=2
Procesando: https://quehacerconlosninos.es/ocio/?e-page-499d516=3
Procesando: https://quehacerconlosninos.es/ocio/?e-page-499d516=272


,nombre_evento,categoría,discapacidad,ubicación,costo,edad_dirigida,min_integrantes,modalidad,url
0,Ocio,N/A,N/A,Madrid,Gratis,Niños/Familia,N/A,Exterior,https://quehacerconlosninos.es/ocio/
1,Ocio,N/A,N/A,Madrid,Gratis,Niños/Familia,N/A,Exterior,https://quehacerconlosninos.es/ocio/?e-page-49...
2,Ocio,N/A,N/A,Madrid,Gratis,Niños/Familia,N/A,Exterior,https://quehacerconlosninos.es/ocio/?e-page-49...
3,Ocio,N/A,N/A,Madrid,Gratis,Niños/Familia,N/A,Exterior,https://quehacerconlosninos.es/ocio/?e-page-49...


## Planes para familia

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica

# Inicializar el driver de Selenium con webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Abrir la página de eventos
url = 'https://planesparafamilias.com/planes/#google_vignette'
driver.get(url)

# Esperar a que los eventos se carguen (ajustamos el selector según la estructura de la página)
try:
    print("Esperando a que los eventos se carguen...")
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-section .elementor-widget-container'))  # Ajustar según la estructura
    )
    print("Los eventos se cargaron correctamente.")
    
    # Extraer los artículos de la página
    eventos = driver.find_elements(By.CSS_SELECTOR, '.elementor-section .elementor-widget-container')  # Ajustar el selector
    print(f"Se encontraron {len(eventos)} eventos.")
    
    # Extraer detalles de cada evento
    for i, evento in enumerate(eventos):
        try:
            # Extraer el título del evento
            titulo = evento.find_element(By.TAG_NAME, 'h3').text.strip() if evento.find_element(By.TAG_NAME, 'h3') else 'Sin título'
            
            # Extraer la descripción del evento (ajustar según lo que encuentres en el HTML)
            descripcion = evento.find_element(By.TAG_NAME, 'p').text.strip() if evento.find_element(By.TAG_NAME, 'p') else 'Sin descripción'
            
            # Imprimir los detalles del evento
            print(f"Evento {i + 1}:")
            print(f"Título: {titulo}")
            print(f"Descripción: {descripcion}")
            print("-" * 50)

        except Exception as e:
            print(f"Error al procesar el evento {i + 1}: {str(e)}")

except Exception as e:
    print(f"Error al esperar la carga de los eventos: {str(e)}")

finally:
    driver.quit()


Esperando a que los eventos se carguen...
Los eventos se cargaron correctamente.
Se encontraron 40 eventos.
Error al procesar el evento 1: Message: no such element: Unable to locate element: {"method":"tag name","selector":"h3"}
  (Session info: chrome=135.0.7049.116); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010fc522c8 chromedriver + 6197960
1   chromedriver                        0x000000010fc498ea chromedriver + 6162666
2   chromedriver                        0x000000010f6cede0 chromedriver + 417248
3   chromedriver                        0x000000010f720797 chromedriver + 751511
4   chromedriver                        0x000000010f7209b1 chromedriver + 752049
5   chromedriver                        0x000000010f714216 chromedriver + 700950
6   chromedriver                        0x000000010f7469ed chromedriver + 907757
7   

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica

# Inicializar el driver de Selenium con webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Abrir la página de eventos
url = 'https://planesparafamilias.com/planes/#google_vignette'
driver.get(url)

# Esperar a que los eventos se carguen (ajustamos el selector según la estructura de la página)
try:
    print("Esperando a que los eventos se carguen...")
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.elementor-section .elementor-widget-container'))  # Ajustar según la estructura
    )
    print("Los eventos se cargaron correctamente.")
    
    # Extraer los artículos de la página
    eventos = driver.find_elements(By.CSS_SELECTOR, '.elementor-section .elementor-widget-container')  # Ajustar el selector
    print(f"Se encontraron {len(eventos)} eventos.")
    
    # Extraer detalles de cada evento
    for i, evento in enumerate(eventos):
        try:
            # Intentar encontrar el título de cada evento
            titulo = evento.find_element(By.CSS_SELECTOR, '.elementor-heading-title').text.strip() if evento.find_element(By.CSS_SELECTOR, '.elementor-heading-title') else 'Sin título'
            
            # Intentar encontrar la descripción del evento
            descripcion = evento.find_element(By.CSS_SELECTOR, 'p').text.strip() if evento.find_element(By.CSS_SELECTOR, 'p') else 'Sin descripción'
            
            # Imprimir los detalles del evento
            print(f"Evento {i + 1}:")
            print(f"Título: {titulo}")
            print(f"Descripción: {descripcion}")
            print("-" * 50)

        except Exception as e:
            print(f"Error al procesar el evento {i + 1}: {str(e)}")

except Exception as e:
    print(f"Error al esperar la carga de los eventos: {str(e)}")

finally:
    driver.quit()


Esperando a que los eventos se carguen...
Los eventos se cargaron correctamente.
Se encontraron 40 eventos.
Error al procesar el evento 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":".elementor-heading-title"}
  (Session info: chrome=135.0.7049.116); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001033742c8 chromedriver + 6197960
1   chromedriver                        0x000000010336b8ea chromedriver + 6162666
2   chromedriver                        0x0000000102df0de0 chromedriver + 417248
3   chromedriver                        0x0000000102e42797 chromedriver + 751511
4   chromedriver                        0x0000000102e429b1 chromedriver + 752049
5   chromedriver                        0x0000000102e36216 chromedriver + 700950
6   chromedriver                        0x0000000102e689ed 

## Es madrid

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL principal
BASE_URL = "https://www.esmadrid.com"
AGENDA_URL = f"{BASE_URL}/agenda-infantil"

# Encabezados para la solicitud HTTP
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Función para obtener los enlaces de los eventos
def obtener_enlaces_eventos():
    res = requests.get(AGENDA_URL, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    enlaces = []

    # Buscar todos los enlaces de eventos en la página
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/") and "/agenda/" in href:
            full_url = BASE_URL + href
            if full_url not in enlaces:
                enlaces.append(full_url)
    return enlaces

# Función para extraer información de cada evento
def extraer_info_evento(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Extraer el título del evento
    titulo_tag = soup.find("h1")
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else "N/A"

    # Extraer la categoría (si está disponible)
    categoria_tag = soup.find("span", class_="field-content")
    categoria = categoria_tag.get_text(strip=True) if categoria_tag else "N/A"

    # Extraer la ubicación (si está disponible)
    ubicacion_tag = soup.find("div", class_="location")
    ubicacion = ubicacion_tag.get_text(strip=True) if ubicacion_tag else "N/A"

    # Extraer la fecha (si está disponible)
    fecha_tag = soup.find("div", class_="date-display-single")
    fecha = fecha_tag.get_text(strip=True) if fecha_tag else "N/A"

    # Extraer el costo (si se menciona "gratis" en el texto)
    texto = soup.get_text().lower()
    costo = "Gratis" if "gratis" in texto else "Pago"

    # Extraer la edad dirigida (si se menciona en el texto)
    if "niños" in texto or "familia" in texto:
        edad_dirigida = "Niños/Familia"
    else:
        edad_dirigida = "N/A"

    # Modalidad (interior o exterior) basada en palabras clave
    if "aire libre" in texto or "parque" in texto:
        modalidad = "Exterior"
    else:
        modalidad = "Interior"

    # Discapacidad (si se menciona accesibilidad en el texto)
    if "accesible" in texto or "silla de ruedas" in texto:
        discapacidad = "Accesible"
    else:
        discapacidad = "N/A"

    # Mínimo de integrantes (si se menciona en el texto)
    min_integrantes = "N/A"  # No se especifica en la mayoría de los casos

    return {
        "nombre_evento": titulo,
        "categoría": categoria,
        "discapacidad": discapacidad,
        "ubicación": ubicacion,
        "fecha": fecha,
        "costo": costo,
        "edad_dirigida": edad_dirigida,
        "min_integrantes": min_integrantes,
        "modalidad": modalidad,
        "url": url
    }

# Obtener los enlaces de los eventos
enlaces_eventos = obtener_enlaces_eventos()

# Extraer información de cada evento
eventos = []
for enlace in enlaces_eventos[:10]:  # Limitar a los primeros 10 eventos
    print(f"Procesando: {enlace}")
    info = extraer_info_evento(enlace)
    eventos.append(info)

# Crear un DataFrame con la información recopilada
df = pd.DataFrame(eventos)

# Mostrar el DataFrame
print(df)

# Opcional: Guardar el DataFrame en un archivo CSV
df.to_csv("eventos_esmadrid_agenda_infantil.csv", index=False)


Procesando: https://www.esmadrid.com/agenda/aladdin-musical-teatro-coliseum
Procesando: https://www.esmadrid.com/agenda/minimaraton-madrid-nebrija-parque-retiro
Procesando: https://www.esmadrid.com/agenda/rana-luna-teatros-luchana
Procesando: https://www.esmadrid.com/agenda/festival-pies-inquietos
Procesando: https://www.esmadrid.com/agenda/teatro-de-titeres-de-el-retiro
Procesando: https://www.esmadrid.com/agenda/saurios-escenario-puerta-angel
Procesando: https://www.esmadrid.com/agenda/funbox-madrid-westfield-parquesur
Procesando: https://www.esmadrid.com/agenda/pequeno-mozart-teatro-lara-sala-candido-lara
Procesando: https://www.esmadrid.com/agenda/wah-madrid-ifema-madrid
Procesando: https://www.esmadrid.com/agenda/suenos-elsa-tributo-frozen-arlequin-gran-via-teatro
                          nombre_evento categoría discapacidad ubicación  \
0                   Aladdín, el musical       N/A          N/A       N/A   
1         VI MiniMaratón Madrid-Nebrija       N/A          N/A      

In [2]:
df

,nombre_evento,categoría,discapacidad,ubicación,fecha,costo,edad_dirigida,min_integrantes,modalidad,url
0,"Aladdín, el musical",N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/aladdin-musica...
1,VI MiniMaratón Madrid-Nebrija,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/minimaraton-ma...
2,Una rana en la Luna,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/rana-luna-teat...
3,3º Festival Pies Inquietos,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/festival-pies-...
4,Teatro de Títeres de El Retiro,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/teatro-de-tite...
5,Saurios,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/saurios-escena...
6,Funbox Madrid,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/funbox-madrid-...
7,El pequeño Mozart,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/pequeno-mozart...
8,WAH Madrid,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/wah-madrid-ife...
9,Los sueños de Elsa. Tributo a Frozen,N/A,N/A,N/A,N/A,Pago,Niños/Familia,N/A,Exterior,https://www.esmadrid.com/agenda/suenos-elsa-tr...


en en cuenta que algunos campos, como "discapacidad", "edad_dirigida" y "min_integrantes", se han estimado basándonos en la información disponible y podrían requerir confirmación adicional.​

## Madrid secreto

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://madridsecreto.co/planes-ninos-madrid/"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

titulos = soup.find_all("h2")

eventos = []

for idx, titulo in enumerate(titulos, start=1):
    nombre = titulo.get_text(strip=True)
    descripcion = ""
    sibling = titulo.find_next_sibling()
    while sibling and sibling.name == "p":
        descripcion += sibling.get_text(strip=True) + " "
        sibling = sibling.find_next_sibling()

    desc_lower = descripcion.lower()

    # Inferencia de categoría
    if any(x in desc_lower for x in ["taller", "crear", "manualidades"]):
        categoria = "talleres"
    elif any(x in desc_lower for x in ["deporte", "barca", "juegos", "parque"]):
        categoria = "deportiva"
    elif any(x in desc_lower for x in ["museo", "historia", "exposición", "espectáculo", "cine", "teatro"]):
        categoria = "cultural"
    elif any(x in desc_lower for x in ["relajación", "bienestar", "salud"]):
        categoria = "salud"
    else:
        categoria = ""

    # Modalidad
    modalidad = "exterior" if any(x in desc_lower for x in ["al aire libre", "parque", "exterior", "jardín"]) else "interior"

    # Costo
    costo = "gratuito" if "gratis" in desc_lower or "gratuito" in desc_lower else "de pago"

    # Edad dirigida
    if "niños pequeños" in desc_lower or "bebés" in desc_lower:
        edad = "0-5 años"
    elif "niños mayores" in desc_lower or "mayores de 6" in desc_lower:
        edad = "6-12 años"
    elif "adolescentes" in desc_lower:
        edad = "13-17 años"
    elif "familia" in desc_lower or "todas las edades" in desc_lower:
        edad = "todas las edades"
    else:
        edad = ""

    # Discapacidad (detección simple de inclusión)
    discapacidad = []
    if any(x in desc_lower for x in ["silla de ruedas", "accesible", "movilidad reducida"]):
        discapacidad.append("motora")
    if any(x in desc_lower for x in ["lengua de signos", "interpretación en lengua", "auditiva"]):
        discapacidad.append("auditiva")
    if any(x in desc_lower for x in ["braille", "tacto", "guías táctiles", "visual"]):
        discapacidad.append("visual")
    if not discapacidad:
        discapacidad = ["ninguna"]

    eventos.append({
        "id_evento": idx,
        "nombre_evento": nombre,
        "categoría": categoria,
        "discapacidad": ", ".join(discapacidad),
        "ubicación": "",  # Se puede estimar según nombre del evento
        "costo": costo,
        "edad_dirigida": edad,
        "min_integrante": "",  # Solo si se menciona explícitamente
        "modalidad": modalidad,
        "descripción": descripcion.strip()
    })

# Crear DataFrame
df = pd.DataFrame(eventos)
df


,id_evento,nombre_evento,categoría,discapacidad,ubicación,costo,edad_dirigida,min_integrante,modalidad,descripción
0,1,1. Títeres del Retiro,deportiva,ninguna,,de pago,,,exterior,Si hay un plan clásico para niños en Madrid so...
1,2,2.Jurassic World: The Experience(a partir del ...,cultural,ninguna,,de pago,,,interior,¿En cuántas casas ocurre que los dinosaurios s...
2,3,3.The FRIENDS™ Experience: The One in Madrid(a...,,ninguna,,de pago,,,interior,"AunqueFRIENDS™fue una serie de los 90,su espír..."
3,4,4. Barcas del lago de la Casa de Campo,deportiva,ninguna,,de pago,,,exterior,Aunque las barcas delparque del Retirotengan m...
4,5,5.Los parques infantiles de Alcobendas,,ninguna,,de pago,,,interior,
5,6,6. El mejor parque infantil de España,,ninguna,,de pago,,,interior,
6,7,"7.DroneArt Show (22, 23, 24 de mayo)",,ninguna,,de pago,,,interior,
7,8,8.Museo Banksy,,ninguna,,de pago,,,interior,Banksyes una figura esencial en el mundo delSt...
8,9,9.Museo del Ferrocarril,,ninguna,,de pago,,,interior,
9,10,10.Una tarde con Frida Kahlo,cultural,ninguna,,de pago,todas las edades,,interior,Acaba de abrir en Madrid una muestra sobre lav...


## Planes familiares

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://planesfamiliares.com"

# Paso 1: Obtener enlaces a eventos
index_url = f"{base_url}/aire-libre/"
resp = requests.get(index_url, headers=headers)
soup = BeautifulSoup(resp.content, "html.parser")

# Encuentra los enlaces a artículos
articulos = soup.select("h3.jeg_post_title > a")
urls_eventos = [a["href"] for a in articulos if a["href"].startswith("http")]

eventos = []

# Paso 2: Scrapeo de cada evento individual
for idx, url in enumerate(urls_eventos, start=1):
    try:
        res = requests.get(url, headers=headers)
        detail = BeautifulSoup(res.content, "html.parser")
        
        titulo = detail.find("h1", class_="jeg_post_title").get_text(strip=True)
        subtitulo = detail.find("h2", class_="jeg_post_subtitle")
        subtitulo = subtitulo.get_text(strip=True) if subtitulo else ""
        desc_lower = (titulo + ". " + subtitulo).lower()

        def inferir_categoria(text):
            if "taller" in text: return "talleres"
            if "deporte" in text: return "deportiva"
            if "museo" in text or "libro" in text: return "cultural"
            return ""

        def inferir_modalidad(text):
            return "exterior" if "retiro" in text or "parque" in text or "aire libre" in text else "interior"

        def inferir_discapacidad(text):
            d = []
            if "silla de ruedas" in text or "accesible" in text: d.append("motora")
            if "lengua de signos" in text: d.append("auditiva")
            if "braille" in text: d.append("visual")
            return d if d else ["ninguna"]

        def inferir_edad(text):
            if "niños pequeños" in text: return "0-5 años"
            if "mayores de 6" in text: return "6-12 años"
            if "adolescentes" in text: return "13-17 años"
            if "familia" in text or "todas las edades" in text: return "todas las edades"
            return ""

        eventos.append({
            "id_evento": idx,
            "nombre_evento": titulo,
            "categoría": inferir_categoria(desc_lower),
            "discapacidad": ", ".join(inferir_discapacidad(desc_lower)),
            "ubicación": "",  # puede inferirse con reglas
            "costo": "gratuito" if "gratis" in desc_lower else "de pago",
            "edad_dirigida": inferir_edad(desc_lower),
            "min_integrante": "",
            "modalidad": inferir_modalidad(desc_lower),
            "min_integrantes": ""
        })

        time.sleep(1)  # Para evitar bloqueos
    except Exception as e:
        print(f"Error con {url}: {e}")

# Convertir a DataFrame
df = pd.DataFrame(eventos)
df


,id_evento,nombre_evento,categoría,discapacidad,ubicación,costo,edad_dirigida,min_integrante,modalidad,min_integrantes
0,1,Faro de Moncloa: vistas que hacen volar la ima...,,ninguna,,de pago,todas las edades,,interior,
1,2,Planes con niños en Villaverde – Abril 2025,,ninguna,,de pago,todas las edades,,interior,
2,3,Planes con niños en Villa de Vallecas – Abril ...,,ninguna,,de pago,todas las edades,,interior,
3,4,Planes con niños en Vicálvaro – Abril 2025,,ninguna,,de pago,todas las edades,,interior,
4,5,Planes con niños en Usera – Abril 2025,,ninguna,,de pago,,,interior,
...,...,...,...,...,...,...,...,...,...,...
57,58,El Tren de Felipe II,,ninguna,,de pago,,,interior,
58,59,Parque de Berlín,,ninguna,,de pago,,,exterior,
59,60,Burrolandia en Madrid,,ninguna,,de pago,,,interior,
60,61,¿Dónde ver los almendros en flor en Madrid?,,ninguna,,de pago,todas las edades,,exterior,


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}
base_url = "http://www.pequeplanning.com"
index_url = f"{base_url}/category/planes/"

response = requests.get(index_url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Encuentra enlaces a los artículos
enlaces = soup.select("h2.entry-title > a")
urls = [a["href"] for a in enlaces]

eventos = []

for idx, url in enumerate(urls, start=1):
    try:
        res = requests.get(url, headers=headers)
        detail = BeautifulSoup(res.content, "html.parser")

        titulo = detail.find("h1", class_="entry-title").get_text(strip=True)
        contenido = detail.find("div", class_="entry-content")
        descripcion = contenido.get_text(separator=" ", strip=True) if contenido else ""
        desc_lower = (titulo + " " + descripcion).lower()

        def inferir_categoria(text):
            if "taller" in text: return "talleres"
            if "deporte" in text or "parque" in text: return "deportiva"
            if "teatro" in text or "museo" in text or "cuento" in text or "obra" in text: return "cultural"
            if "yoga" in text or "relajación" in text: return "salud"
            return ""

        def inferir_modalidad(text):
            return "exterior" if "aire libre" in text or "parque" in text else "interior"

        def inferir_discapacidad(text):
            d = []
            if "accesible" in text or "movilidad reducida" in text: d.append("motora")
            if "lengua de signos" in text: d.append("auditiva")
            if "braille" in text or "guía táctil" in text: d.append("visual")
            return d if d else ["ninguna"]

        def inferir_edad(text):
            if "bebés" in text or "menores de 3" in text: return "0-3 años"
            if "peques" in text or "niños" in text: return "4-12 años"
            if "adolescentes" in text: return "13-17 años"
            if "familia" in text or "para todos": return "todas las edades"
            return ""

        eventos.append({
            "id_evento": idx,
            "nombre_evento": titulo,
            "categoría": inferir_categoria(desc_lower),
            "discapacidad": ", ".join(inferir_discapacidad(desc_lower)),
            "ubicación": "",  # Puedes inferir por reglas
            "costo": "gratuito" if "gratis" in desc_lower else "de pago",
            "edad_dirigida": inferir_edad(desc_lower),
            "min_integrante": "",
            "modalidad": inferir_modalidad(desc_lower),
            "min_integrantes": ""
        })

        time.sleep(1)  # para no saturar el servidor
    except Exception as e:
        print(f"Error en {url}: {e}")

# DataFrame final
df = pd.DataFrame(eventos)
df


,id_evento,nombre_evento,categoría,discapacidad,ubicación,costo,edad_dirigida,min_integrante,modalidad,min_integrantes
0,1,Nueva temporada de las fuentes de La Granja de...,,ninguna,,de pago,todas las edades,,interior,
1,2,"Talleres gratuitos para familias, los sábados ...",talleres,ninguna,,de pago,todas las edades,,interior,
2,3,Arranca la temporada de primavera del Tren de ...,,ninguna,,de pago,todas las edades,,interior,
3,4,La exposición “Saurios” trae más de 100 dinosu...,,ninguna,,de pago,todas las edades,,interior,
4,5,Nueva edición de Teatralia: 111 funciones de 2...,,ninguna,,de pago,todas las edades,,interior,
5,6,Festival de circo gratuito en Torrejón de Ardo...,,ninguna,,de pago,todas las edades,,interior,
6,7,Matadero acoge una instalación gratuita de jue...,,ninguna,,de pago,todas las edades,,interior,
7,8,Temporada de primavera del “Tren de Arganda”,,ninguna,,de pago,todas las edades,,interior,
8,9,"Llega el Carnaval 2025, con epicentro en Madri...",,ninguna,,de pago,todas las edades,,interior,
9,10,La Feria Medieval de Chinchón también para niños,,ninguna,,de pago,4-12 años,,interior,
